# Our data is present in form of directories and sub-directories...and this block will extract face from each image and then convert that data into a single numpy array and store in a compressed file.

In [2]:
from os import listdir
from os.path import isdir
import numpy as np;
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from PIL import Image
import cv2;

detector = MTCNN()

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
     image = Image.open(filename)
     image = image.convert('RGB')
     pixels = asarray(image)
     results = detector.detect_faces(pixels)
     if len(results)==0:
        return (False,None);
     x1, y1, width, height = results[0]['box']
     x1, y1 = abs(x1), abs(y1)
     x2, y2 = x1 + width, y1 + height
     face = pixels[y1:y2, x1:x2]
     image = Image.fromarray(face)
     image = image.resize(required_size)
     face_array = asarray(image)
     return (True,face_array);
 
    
# load images and extract faces for all images in a directory
def load_faces(directory):
     faces = [];
     if(directory.find('.DS_Store')!=-1):
      return faces;
     for filename in listdir(directory):
         if(filename.find('.DS_Store')!=-1): # if it finds a match
            continue;
         path = directory + filename
         flag,face = extract_face(path)
         if(flag):
             faces.append(face)
     return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
     X, y = [],[]
     for subdir in listdir(directory):
         path = directory + '/'+ subdir + '/'
         faces = load_faces(path)
         labels = [subdir for _ in range(len(faces))]
         print(labels);
         X.extend(faces)
         y.extend(labels)
     return asarray(X), asarray(y)

trainX, trainy = load_dataset('Face Dataset/train/')
print(trainX.shape, trainy.shape)
testX, testy = load_dataset('Face Dataset/test/')
savez_compressed('Face Dataset.npz', trainX, trainy, testX, testy)

['Abhishek rollno-1', 'Abhishek rollno-1', 'Abhishek rollno-1']
['Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya']
['Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply']
['Kushal Kumar Mehra', 'Kushal Kumar Mehra', 'Kushal Kumar Mehra', 'Kushal Kumar Mehra']
[]
['Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna']
['Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge']
['Pawan', 'Pawan', 'Pawan']
['Kamesh', 'Kamesh', 'Kamesh', 'Kamesh', 'Kamesh']
['Krishan', 'Krishan', 'Krishan', 'Krishan', 'Krishan']
['Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta']
['Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat']
['Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra']
['Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak']
['Jati

In [3]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet
from keras.models import load_model

    
def get_embedding(model, face_pixels):
 face_pixels = face_pixels.astype('float32')
#  mean, std = face_pixels.mean(), face_pixels.std()
#  face_pixels = (face_pixels - mean) / std
 face_pixels=face_pixels.reshape(1,160,160,3);
 yhat = model.embeddings(face_pixels)
 return yhat[0]
 

    
data = load('Face Dataset.npz',allow_pickle=True)
model = FaceNet() 
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
newTrainX = [];
for face_pixels in trainX:
 embedding = get_embedding(model, face_pixels)
 newTrainX.append(embedding)
newTrainX = asarray(newTrainX)


newTestX = [];
for face_pixels in testX:
 embedding = get_embedding(model, face_pixels)
 newTestX.append(embedding)
newTestX = asarray(newTestX)

savez_compressed('Face Dataset_embeddings.npz', newTrainX, trainy, newTestX, testy)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
data = load('Face Dataset_embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy_encoded = out_encoder.transform(trainy)
testy_encoded = out_encoder.transform(testy)
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy_encoded)
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
print(yhat_train);
print(yhat_test)
# score
score_train = accuracy_score(trainy_encoded, yhat_train)
score_test = accuracy_score(testy_encoded, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

[ 1  1  1 21 21 21 21 21 25 25 25 25 25 18 18 18 18 28 28 28 28 28 12 12
 12 12 12 26 26 26 16 16 16 16 16 17 17 17 17 17 10 10 10 10 10 23 23 23
 23 23 27 27 27 27 27  4  4  4  4  4 15 15 15 15 15 13 13 13 13 13  8  8
  8  8  8 20 20 20 20 20  2  2 24 24 24 24 24 22 22 22 22 22  3  3  3  3
  3  6  6  6  6 19 19 19 19 14 14 14 14 14  5  5  5  5  5  7  7  7  7  7
 30 30 30 30 30 29 29 29 29 29  9  9  9  9  0  0  0  0  0 11 11 11 11 11]
[21 21 25 25 18 16 18 28 28 12 12 26 16 17 17 17 17 17 17 17 10 23 23 23
 23 23 23 23 23 23 23 27 27 27 27 27 27 27  4  4  4 15 13 13  8  8  8  8
 20 20 24 24 24 22 22 22  3  3  3  3  6 19 14 14 14 14 14 14 14  5  5  5
  5  7  7  7 30 30 30 12 30 30 29 29 29  9  0 11 11 11]
Accuracy: train=100.000, test=97.778


In [15]:
for i in range(0,len(yhat_test)):
    if(yhat_test[i]!=testy_encoded[i]):
        print(i,' ',testy[i])

5   Kushal Kumar Mehra
79   Yatish Singhal


In [17]:
from os import listdir
from os.path import isdir
import numpy as np;
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from PIL import Image
import cv2;
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
data = load('Face Dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'];
np.linalg.norm(trainX[1] - trainX[30])

1.4842321

In [22]:
for image in trainX:
    print(np.linalg.norm(trainX[24] - image))
print(trainy[0])

1.4424807
1.5859622
1.3932127
1.4572102
1.486579
1.4300975
1.3472986
1.5176482
1.4472355
1.40514
1.3550465
1.5328306
1.4372666
1.4863945
1.4452046
1.2327849
1.2471089
1.3049556
1.2173941
1.3790109
1.3680788
0.4906456
0.6482883
0.5817074
0.0
0.582185
0.38642964
0.7135611
1.0466834
1.1032581
1.1858091
1.0333297
1.1440543
1.107784
1.1221755
1.1394687
1.1077505
1.1181142
1.0734725
1.2084377
1.27036
1.2571555
1.2703248
1.2910684
1.1249479
1.2997386
1.2878506
1.3306391
1.3255755
1.308179
1.2435036
1.236963
1.1082865
1.2285538
1.3330797
1.147328
1.1178598
1.0233992
1.2216623
1.2911412
1.2364049
1.2337321
1.230654
1.1957871
1.108449
1.2742175
1.2654743
1.3452821
1.360522
1.3707945
1.3987929
1.3402373
1.3544977
1.345175
1.247825
1.2416341
1.2305913
1.2925004
1.2786534
1.2687876
1.4140908
1.4693819
1.5114418
1.5086243
1.4498621
1.5047351
1.5067531
1.3218526
1.4504921
1.4404182
1.5605661
1.4754181
1.4573054
1.4440656
1.2936515
1.3617352
1.174803
1.1560627
1.2202296
1.2409858
1.3017288
1.3221021
1